In [ ]:
from google.colab import files

uploaded = files.upload()  # This will prompt you to select files from your computer


Saving nebius_api_key to nebius_api_key
Saving openai_api_key to openai_api_key


In [ ]:
with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

with open("openai_api_key", "r") as file:
    openai_api_key = file.read().strip()

In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok


In [ ]:
import os
import nest_asyncio
from pyngrok import ngrok
import uvicorn

##### This is for Image support and generating using prompt
from PIL import Image
from io import BytesIO
import base64
import json
import matplotlib.pyplot as plt

# Set your keys here (for Colab, use os.environ or set directly)
os.environ["NEBIUS_API_KEY"] = nebius_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

from openai import OpenAI
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY")
)

# Copy generate_image, describe_image and broken_telephone from previous exercise into here

def generate_image(prompt):
    response = client.images.generate(         # this line will return an ImagesResponse object, not a Python dictionary. Need to use JSON method to parse
        model="black-forest-labs/flux-dev",
        prompt=prompt,
        n=1,
        size="512x512",
        response_format="b64_json"
    )
    response_json = response.to_json()  # Get JSON string
    response_data = json.loads(response_json)  # Convert to dict
    img_data = response_data['data'][0]['b64_json']
    img = Image.open(BytesIO(base64.b64decode(img_data)))
    return img

# Action 2: Create function to describe image
def describe_image(img):
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_b64 = base64.b64encode(buffered.getvalue()).decode()

    messages = [
        {"role": "user", "content": [
            {"type": "text", "text": "Describe this image in one sentence."},
            {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64}"}}
        ]}
    ]

    response = client.chat.completions.create(
        model="Qwen/Qwen2-VL-72B-Instruct",
        messages=messages,
        max_tokens=60,
    )
    return response.choices[0].message.content


# Action 3: Create loop to loop image and decibing image based on number of rounds I want when I call this function
def broken_telephone(starting_prompt, n_rounds):
    history = []  # To store tuples of (text, image) Action 6: Define history variable
    current_text = starting_prompt
    print(f"Round 0 (Start Prompt): {current_text}")

    for round_num in range(1, n_rounds + 1):
        # Step 1: Generate an image from the current text
        img = generate_image(current_text)
        img.show()  # Optional: to see each image
        img.save(f"round_{round_num}.png")  # Save each round's image for reference
        history.append((None, img))  # Store image only. Action 4: Added this code to collect all history of images

        # Step 2: Describe the generated image to produce new text
        current_text = describe_image(img)
        print(f"Round {round_num} (Generated Description): {current_text}")
        history.append((current_text, None))  # Store description only. Action 5: Added this code to collect all history of descriptions

    print("Final Output:", current_text)

    # Action 6: Print all history I collected earlier
    print("\n--- Summary of All Rounds ---")
    for i, (text, image) in enumerate(history):
        if text:
            print(f"Round {i}: TEXT: {text}")
        if image:
            print(f"Round {i}: IMAGE: [Saved as round_{i//2 + 1}.png]")


    # Optionally: Return history for further analysis
    return history

# ------------------------FAST API------------------------------------------------
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class PromptRequest(BaseModel):
    prompt: str
    rounds: int = 3

# Creates an HTTP POST endpoint at /broken-telephone
@app.post("/broken-telephone")

def run_broken_telephone(request: PromptRequest):         # Accepts a JSON body matching PromptRequest.
    # Replace with your actual broken_telephone function
    result = broken_telephone(request.prompt, request.rounds)

    # For demo, just return the list of descriptions
    descriptions = [text for (text, img) in result if text]
    return {"history": descriptions}

# --- Launch FastAPI + ngrok ---
nest_asyncio.apply()

# Add your ngrok authtoken here. Replace 'YOUR_NGROK_AUTHTOKEN' with your actual token.
# You can get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
ngrok.set_auth_token("2xE01tzXmG4HV3ladzd4cRPEw8E_CtStC7Hm5RPH3M1VQzJE")

# Expose the app on port 8000
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the app (in background)
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://2776-34-46-208-119.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [1174]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Round 0 (Start Prompt): A robot playing the piano on the moon
INFO:     45.195.232.170:0 - "POST /broken-telephone HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",